In [ ]:
!git clone https://github.com/Campus-24/Image_Classification.git

Cloning into 'Image_Classification'...
remote: Enumerating objects: 6253, done.
remote: Total 6253 (delta 0), reused 0 (delta 0), pack-reused 6253
Receiving objects: 100% (6253/6253), 573.95 MiB | 20.34 MiB/s, done.
Resolving deltas: 100% (56/56), done.
Checking out files: 100% (5916/5916), done.


In [ ]:
!ls

In [ ]:
!ls Image_Classification

frozen_east_text_detection.pb  Image_dataset  LICENSE  README.md


In [ ]:
from imutils.object_detection import non_max_suppression
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow # for image display
from skimage import io
from PIL import Image 
import time
import os

In [ ]:
count=0                                             #for totalcount of images
counttext=0                                         #for count of images  detected correctly 
min_confidence=0.9999
directory="/content/Image_Classification/Image_dataset/images_text"            #directory of images with text

#detecting images with text and  finding count  of them
for i in os.scandir(directory):
  count=count+1
  image_raw=cv.imread(i.path)
  image=cv.cvtColor(image_raw,cv.COLOR_BGR2RGB)      #loading image and finding image dimensions
  orig=image.copy()
  (H,W)=image.shape[:2]

  # set the new width and height and then determine the ratio in change
  # for both the width and height
  (newW,newH)=(320,320)
  # resize the image and grab the new image dimensions
  image = cv.resize(image, (newW, newH))
  (H, W) = image.shape[:2]

  #define the two output layer names for the EAST detector model that
  # we are interested -- the first is the output probabilities and the
  # second can be used to derive the bounding box coordinates of text
  layerNames = [
	"feature_fusion/Conv_7/Sigmoid",
	"feature_fusion/concat_3"]

  # load the pre-trained EAST text detector
  print("[INFO] loading EAST text detector...")
  net = cv.dnn.readNet("/content/Image_Classification/frozen_east_text_detection.pb")

  # construct a blob from the image and then perform a forward pass of
  # the model to obtain the two output layer sets
  blob = cv.dnn.blobFromImage(image, 1.0, (W, H),(123.68, 116.78, 103.94), swapRB=True, crop=False)
  net.setInput(blob)
  (scores, geometry) = net.forward(layerNames)

  # grab the number of rows and columns from the scores volume, then
  # initialize our set of bounding box rectangles and corresponding
  # confidence scores
  (numRows, numCols) = scores.shape[2:4]
  rects = []
  confidences = []

  # loop over the number of rows
  for y in range(0, numRows):
    # extract the scores (probabilities), followed by the geometrical
    # data used to derive potential bounding box coordinates that
    # surround text
    scoresData = scores[0, 0, y]
    xData0 = geometry[0, 0, y]
    xData1 = geometry[0, 1, y]
    xData2 = geometry[0, 2, y]
    xData3 = geometry[0, 3, y]
    anglesData = geometry[0, 4, y]
    
    # loop over the number of columns
    for x in range(0, numCols):

      # if our score does not have sufficient probability, ignore it
      if scoresData[x] < min_confidence:
        continue
      # compute the offset factor as our resulting feature maps will
      # be 4x smaller than the input image  
      (offsetX, offsetY) = (x * 4.0, y * 4.0)
      angle = anglesData[x]
      cos = np.cos(angle)
      sin = np.sin(angle)
      
      # use the geometry volume to derive the width and height of
      # the bounding box
      h = xData0[x] + xData2[x]
      w = xData1[x] + xData3[x]

      # compute both the starting and ending (x, y)-coordinates for
      # the text prediction bounding box
      endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
      endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
      startX = int(endX - w)
      startY = int(endY - h)
      # add the bounding box coordinates and probability score to
      # our respective lists
      rects.append((startX, startY, endX, endY))
      confidences.append(scoresData[x])
  check=0
  
  #finding images  with bounding boxes having more than  0.9999 of confidence, having found atleast one, the loop breaks and marks the  image one having text
  for i in confidences:
    if i>0.99:
      print("image has text")
      check=1
      counttext=counttext+1
      break
    else:
      continue
  #if none of the bounding boxes are having confidence of having text more than 0.9999, image is marked of not having text     
  if check == 0:
    print("image has no text")




Streaming output truncated to the last 5000 lines.
image has text
[INFO] loading EAST text detector...
[INFO] text detection took 0.631080 seconds
image has text
[INFO] loading EAST text detector...
[INFO] text detection took 0.655113 seconds
image has text
[INFO] loading EAST text detector...
[INFO] text detection took 0.642377 seconds
image has text
[INFO] loading EAST text detector...
[INFO] text detection took 0.633886 seconds
image has text
[INFO] loading EAST text detector...
[INFO] text detection took 0.632094 seconds
image has no text
[INFO] loading EAST text detector...
[INFO] text detection took 0.635108 seconds
image has text
[INFO] loading EAST text detector...
[INFO] text detection took 0.633380 seconds
image has text
[INFO] loading EAST text detector...
[INFO] text detection took 0.629416 seconds
image has no text
[INFO] loading EAST text detector...
[INFO] text detection took 0.620029 seconds
image has text
[INFO] loading EAST text detector...
[INFO] text detection took 

TypeError: ignored

In [ ]:
print("Accuracy:  {} identified correctly from {}".format(counttext,count))
print("Accuracy Percentage: {}".format((counttext/count)*100))

Accuracy:  2804 identified correctly from 3227
Accuracy Percentage: 86.89185001549427


# Subject wise image classification. (applied on the non texual images)


In [ ]:
# Getting the filenames from the folder 'Notext':
import os
filenames=os.listdir('/content/Image_Classification/Image_dataset/images_nontext')
print(filenames)

['66448.jpg', '31882.jpg', '50268.jpg', '16079.jpg', '48120.jpg', '64632.jpg', '25356.jpg', '50486.jpg', '53175.jpg', '3628.jpg', '55419.jpg', '52788.jpg', '20413.jpg', '57957.jpg', '7167.jpg', '47068.jpg', '72394.jpg', '26420.jpg', '18087.jpg', '38742.jpg', '37904.jpg', '38104.jpg', '52863.jpg', '63882.jpg', '24434.jpg', '40520.jpg', '3062.jpg', '72748.jpg', '50292.jpg', '67872.jpg', '3134.jpg', '63784.jpg', '45004.jpg', '63833.jpg', '53629.jpg', '30494.jpg', '10258.jpg', '59702.jpg', '33557.jpg', '29287.jpg', '16723.jpg', '70248.jpg', '72965.jpg', '62774.jpg', '44511.jpg', '22334.jpg', '66224.jpg', '16019.jpg', '40407.jpg', '44187.jpg', '32374.jpg', '17007.jpg', '13558.jpg', '70369.jpg', '49450.jpg', '19523.jpg', '33547.jpg', '72498.jpg', '49470.jpg', '22983.jpg', '15565.jpg', '3473.jpg', '16890.jpg', '56403.jpg', '56256.jpg', '66941.jpg', '4302.jpg', '57895.jpg', '64069.jpg', '5740.jpg', '50774.jpg', '40197.jpg', '16385.jpg', '54424.jpg', '30627.jpg', '30438.jpg', '68967.jpg', '6828

In [ ]:
%cd '/content/Image_Classification/Image_dataset/images_nontext'

/content/Image_Classification/Image_dataset/images_nontext


In [ ]:
%ls

10000.jpg  24417.jpg  33950.jpg  45901.jpg  56522.jpg  65954.jpg
1000.jpg   24420.jpg  3400.jpg   45934.jpg  56595.jpg  66000.jpg
1001.jpg   24421.jpg  34016.jpg  45966.jpg  56619.jpg  66001.jpg
10038.jpg  24427.jpg  34030.jpg  45989.jpg  56631.jpg  66015.jpg
10068.jpg  24434.jpg  34074.jpg  46002.jpg  56632.jpg  66050.jpg
10142.jpg  24435.jpg  34083.jpg  46107.jpg  56701.jpg  66051.jpg
10208.jpg  24512.jpg  34095.jpg  46159.jpg  56714.jpg  66063.jpg
1021.jpg   24530.jpg  34123.jpg  46195.jpg  56732.jpg  66073.jpg
10258.jpg  24559.jpg  34124.jpg  46231.jpg  56747.jpg  66114.jpg
10270.jpg  2455.jpg   34133.jpg  46283.jpg  56819.jpg  66138.jpg
10318.jpg  24562.jpg  3413.jpg   46313.jpg  56845.jpg  66205.jpg
10343.jpg  24568.jpg  34168.jpg  46382.jpg  56860.jpg  66212.jpg
10357.jpg  24590.jpg  34174.jpg  46396.jpg  56884.jpg  66224.jpg
10358.jpg  24594.jpg  34190.jpg  46407.jpg  5689.jpg   66232.jpg
10369.jpg  24606.jpg  34251.jpg  46439.jpg  56954.jpg  66270.jpg
10402.jpg  24632.jpg  342

In [ ]:
# Importing the necessary libraries:
import numpy as np
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.imagenet_utils import decode_predictions
import matplotlib.pyplot as plt

In [ ]:
# Downloading the pretrained resnet model:
model = ResNet50(weights='imagenet')

In [ ]:
img=[]
for i in range(len(filenames)):
  img.append(load_img(filenames[i], target_size=(224, 224)))          # Loading the filenames. (224, 224) is necessary because resnet takes input in this format.
  img[i] = img_to_array(img[i])                                       # Converting images to array. 
  img[i] = np.expand_dims(img[i], axis=0)                             
  img[i] = preprocess_input(img[i])                                   # Preprocessing the input.
  preds = model.predict(img[i])                                       # Predicting the classes.
  print('Predicted:', decode_predictions(preds, top=1)[0])            # Prediction.

Predicted: [('n04507155', 'umbrella', 0.2052832)]
Predicted: [('n03594734', 'jean', 0.31883135)]
Predicted: [('n09332890', 'lakeside', 0.44968656)]
Predicted: [('n04409515', 'tennis_ball', 0.34363994)]
Predicted: [('n09229709', 'bubble', 0.4071962)]
Predicted: [('n04596742', 'wok', 0.37053132)]
Predicted: [('n04456115', 'torch', 0.5673703)]
Predicted: [('n04005630', 'prison', 0.46802288)]
Predicted: [('n03877472', 'pajama', 0.125467)]
Predicted: [('n04584207', 'wig', 0.22736694)]
Predicted: [('n06359193', 'web_site', 0.74485797)]
Predicted: [('n07579787', 'plate', 0.56867236)]
Predicted: [('n02356798', 'fox_squirrel', 0.29502162)]
Predicted: [('n03933933', 'pier', 0.6934489)]
Predicted: [('n04550184', 'wardrobe', 0.30665648)]
Predicted: [('n07248320', 'book_jacket', 0.123340346)]
Predicted: [('n03782006', 'monitor', 0.36724514)]
Predicted: [('n04356056', 'sunglasses', 0.48530608)]
Predicted: [('n04371774', 'swing', 0.3410204)]
Predicted: [('n03710193', 'mailbox', 0.20206001)]
Predicted

IsADirectoryError: ignored